In [3]:
%%capture
!pip install topic-wizard
!pip install sklearn

In [2]:
import topicwizard
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [1]:
from random import sample

In [3]:
# Downloading newsgroups dataset
corpus = fetch_20newsgroups(subset="all").data

In [5]:
# Setting up topic modelling pipeline
vectorizer = CountVectorizer(min_df=10, max_df=0.5)
# NMF topic model with 10 topics
nmf = NMF(n_components=10)
pipeline = Pipeline([
    ("bow", vectorizer),
    ("nmf", nmf)
])

In [6]:
# Fitting pipeline
pipeline.fit(corpus)

Pipeline(steps=[('bow', CountVectorizer(max_df=0.5, min_df=10)),
                ('nmf', NMF(n_components=10))])

In [10]:
vectorizer.get_feature_names_out().shape[0]

4478

In [9]:
# Visualizing with topicwizard
topicwizard.visualize(pipeline=pipeline, corpus=corpus, port=8051)

Preprocessing
Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/



INFO:topicwizard.app:Dash is running on http://127.0.0.1:8051/



 * Serving Flask app 'topicwizard.app'
 * Debug mode: off
